In [1]:
from datetime import datetime
from dateutil import parser

from nltk.corpus import stopwords
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import math
from collections import Counter
import numpy as np


import re

### Import Data

In [2]:
joe_df = pd.read_csv('joe.csv', parse_dates=True)#,index_col='date')
joe_df['date_time']= pd.to_datetime(joe_df['date'])
joe_df['date']= joe_df['date_time'].dt.date
joe_df.head()

,Unnamed: 0,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,date_time
0,0,1325885871875190784,1325885871875190784,1.604951e+12,2020-11-09,-500,NaN,The bottom line: I will spare no effort to tur...,en,[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,2020-11-09 14:40:00
1,1,1325880083618426881,1325880083618426881,1.604949e+12,2020-11-09,-500,NaN,The challenge before us right now is still imm...,en,[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,2020-11-09 14:17:00
2,2,1325873288711712769,1325873288711712769,1.604948e+12,2020-11-09,-500,NaN,My COVID-19 Transition Advisory Board will adv...,en,[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,2020-11-09 13:50:00
3,3,1325870017401905152,1325870017401905152,1.604947e+12,2020-11-09,-500,NaN,"Today, I have named a COVID-19 Transition Advi...",en,[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,2020-11-09 13:37:00
4,4,1325842292444291072,1325842292444291072,1.604940e+12,2020-11-09,-500,NaN,I spent the morning with the co-chairs of my C...,en,[],...,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,2020-11-09 11:46:50


### Define Preprocessing Function

In [3]:
#remove urls
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

#filter stop words
stop_words = set(stopwords.words('english'))

def remove_stop_words(txt_list):
    filtered_txt_list = []
    for word in txt_list:
        if word not in stop_words:
            filtered_txt_list.append(word)
    return(filtered_txt_list)

#compute TF
def computeTF(x):
    wordList= x[0]
    wordFreq =x[1]
    tfDict = {}
    bagOfWordsCount = len(wordList)
    for word, count in wordFreq.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return Counter(tfDict)


#computer term_counter
def term_counter(txt_list):
    term_counts = {}
    for i in set(txt_list):
        term_counts[i] = txt_list.count(i)
    return(Counter(term_counts))

#return list of term counts for each tweet
def return_term_count_list(df, term_count_column = "term_count"):
    term_count_list = []
    for i in df[term_count_column]:
        term_count_list.append(dict(i))
    return(term_count_list)


#return copurs set for all tweets
def return_corpus_set(df, term_list_column = "term_list"):    
    tweet_corpus = []
    for i in df[term_list_column]:
        tweet_corpus += i
    corpus_set = set(tweet_corpus)
    return(corpus_set)

#computer IDF
def computeIDF(documents,corpus_set):
    N = len(documents)

    idfDict = dict.fromkeys(corpus_set, 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1


    for word, val in idfDict.items():
        idfDict[word] = math.log(N / (float(val))) #smoothing?
    return(Counter(idfDict))

#computes TFIDF
def computeTFIDF(TF,idfs):
    tfidf = {}
    for word, val in TF.items():
        tfidf[word] = val * idfs[word]
    return(Counter(tfidf))

In [4]:
def TFIDF_processing(df, startdate = "August 11, 2020"):
    df = df[["id","date_time","date","tweet"]]

    
    #filter data frame:
    startdate = parser.parse(str(startdate))
    df = df[df["date_time"] > startdate]
    df['term_list'] = df['tweet'].apply(lambda x: remove_url(x))
    df['term_list'] = df['term_list'].apply(lambda x: x.lower().split())
    df['term_list'] = df['term_list'].apply(lambda x: remove_stop_words(x))
    
    #calc TFs
    df['term_count'] = df['term_list'].apply(lambda x: term_counter(x))
    df["TF"] = df[['term_list','term_count']].apply(computeTF,axis=1)
    
    #return corpus and doc list
    term_count_list_df = return_term_count_list(df)
    corpus_set_df = return_corpus_set(df)
    
    #calc IDF and  TF-IDF
    df_idf = computeIDF(term_count_list_df,corpus_set_df)
    df["TFIDF"] = df["TF"].apply(lambda x: computeTFIDF(x, df_idf))
    
    return(df, df_idf, corpus_set_df)

In [5]:
joe_df, joe_idf,joe_corpus = TFIDF_processing(joe_df)
joe_df

,id,date_time,date,tweet,term_list,term_count,TF,TFIDF
0,1325885871875190784,2020-11-09 14:40:00,2020-11-09,The bottom line: I will spare no effort to tur...,"[bottom, line, spare, effort, turn, pandemic, ...","{'line': 1, 'turn': 1, 'spare': 1, 'effort': 1...","{'line': 0.14285714285714285, 'turn': 0.142857...","{'line': 0.6429432866213801, 'turn': 0.7419643..."
1,1325880083618426881,2020-11-09 14:17:00,2020-11-09,The challenge before us right now is still imm...,"[challenge, us, right, still, immense, growing...","{'right': 1, 'us': 1, 'challenge': 1, 'pandemi...","{'right': 0.09090909090909091, 'us': 0.0909090...","{'right': 0.3092718834255956, 'us': 0.19981981..."
2,1325873288711712769,2020-11-09 13:50:00,2020-11-09,My COVID-19 Transition Advisory Board will adv...,"[covid19, transition, advisory, board, advise,...","{'empathy': 1, 'science': 1, 'advisory': 1, 'e...","{'empathy': 0.058823529411764705, 'science': 0...","{'empathy': 0.35535576748804765, 'science': 0...."
3,1325870017401905152,2020-11-09 13:37:00,2020-11-09,"Today, I have named a COVID-19 Transition Advi...","[today, named, covid19, transition, advisory, ...","{'advisory': 1, 'plan': 1, 'public': 1, 'put':...","{'advisory': 0.04, 'plan': 0.04, 'public': 0.0...","{'advisory': 0.24164192189187242, 'plan': 0.12..."
4,1325842292444291072,2020-11-09 11:46:50,2020-11-09,I spent the morning with the co-chairs of my C...,"[spent, morning, cochairs, covid19, council, d...","{'forward': 1, 'morning': 1, 'covid19': 1, 'di...","{'forward': 0.0625, 'morning': 0.0625, 'covid1...","{'forward': 0.29092210538605745, 'morning': 0...."
...,...,...,...,...,...,...,...,...
1256,1293340421444124672,2020-08-11 20:16:00,2020-08-11,"Let’s go win this, @KamalaHarris. https://t.c...","[lets, go, win, kamalaharris]","{'kamalaharris': 1, 'win': 1, 'go': 1, 'lets': 1}","{'kamalaharris': 0.25, 'win': 0.25, 'go': 0.25...","{'kamalaharris': 0.695737877318832, 'win': 0.8..."
1257,1293280412144267264,2020-08-11 16:17:32,2020-08-11,"Back when Kamala was Attorney General, she wor...","[back, kamala, attorney, general, worked, clos...","{'lifted': 1, 'working': 1, 'beau': 1, 'closel...","{'lifted': 0.043478260869565216, 'working': 0....","{'lifted': 0.2802831806697815, 'working': 0.17..."
1258,1293280411150217219,2020-08-11 16:17:32,2020-08-11,I have the great honor to announce that I’ve p...,"[great, honor, announce, ive, picked, kamalaha...","{'fearless': 1, 'great': 1, 'kamalaharris': 1,...","{'fearless': 0.058823529411764705, 'great': 0....","{'fearless': 0.4199800197626423, 'great': 0.25..."
1259,1293215600680951810,2020-08-11 12:00:00,2020-08-11,Don't forget that in the middle of this pandem...,"[dont, forget, middle, pandemic, trump, admini...","{'middle': 1, 'build': 1, 'insurance': 1, 'try...","{'middle': 0.04, 'build': 0.04, 'insurance': 0...","{'middle': 0.16575712249643715, 'build': 0.121..."


In [6]:
def sum_normalize_collections(counter_list):
    counter_sum = Counter()
    for counter in counter_list:
        counter_sum += counter
    for k,v in counter_sum.items():
        counter_sum[k] = v/len(counter_list)
    return(counter_sum.most_common())

In [7]:
group_by_series = joe_df.groupby("date")["TFIDF"].apply(lambda x:x.to_list())
group_by_series

date
2020-08-11    [{'join': 0.2940618686100579, 'ever': 0.337384...
2020-08-12    [{'kamalaharris': 0.16370302995737224, 'called...
2020-08-13    [{'82': 3.56983016798246, 'days': 1.5661635753...
2020-08-14    [{'kamalaharris': 0.695737877318832, 'americas...
2020-08-15    [{'president': 0.21785772830962136, 'ready': 0...
                                    ...                        
2020-11-04    [{'across': 0.12819222485363846, 'american': 0...
2020-11-05    [{'votes': 0.23251742898855932, 'counted': 0.4...
2020-11-06    [{'opponents': 1.6116282888512437, 'may': 1.00...
2020-11-07    [{'thank': 1.224641477721731, 'bottom': 2.0136...
2020-11-09    [{'line': 0.6429432866213801, 'turn': 0.741964...
Name: TFIDF, Length: 90, dtype: object

In [8]:
score_df = pd.DataFrame(joe_df.groupby("date")["TFIDF"].apply(lambda x: sum_normalize_collections(x.to_list())))
score_df

,TFIDF
date,
2020-08-11,"[(win, 0.18158107361427933), (go, 0.1437825367..."
2020-08-12,"[(kamalaharris, 0.13117880488284), (worried, 0..."
2020-08-13,"[(82, 0.39664779644249554), (days, 0.174018175..."
2020-08-14,"[(story, 0.2673950433368432), (social, 0.23918..."
2020-08-15,"[(know, 0.11188818823996126), (vice, 0.0985908..."
...,...
2020-11-04,"[(count, 0.145789478930796), (victory, 0.13931..."
2020-11-05,"[(counted, 0.29949649183368193), (faith, 0.121..."
2020-11-06,"[(opponents, 0.2302326126930348), (enemies, 0...."


In [9]:
# Document Vectorization
def TFIDF_vecorization(df,corpus):
    D = np.zeros((len(df), len(corpus)))

    for i,tfidf in enumerate(df["TFIDF"]):
        for term in tfidf:
            ind = (list(corpus)).index(term)
            D[i][ind] = tfidf[term]
    return(D)

### Next Steps:
#### clean data:
-handle punctuation/concatenated words (first pass done)
-may pick specific work fileter? ( )
#### average TIDF scores by day
-vectorize TIDF dictionaries ? (wrote a function, not really ideal)

-sum TF-IDF vector with in a day, normalize to number of tweets with in that day (done)

-Return top 50 scores for that done ( )